In [1]:
import pandas as pd
import langid
pd.set_option("display.max_columns", 50)

In [2]:
df = pd.read_csv("goodreads_books.csv")
df.head()

,id,title,link,series,cover_link,author,author_link,rating_count,review_count,average_rating,five_star_ratings,four_star_ratings,three_star_ratings,two_star_ratings,one_star_ratings,number_of_pages,date_published,publisher,original_title,genre_and_votes,isbn,isbn13,asin,settings,characters,awards,amazon_redirect_link,worldcat_redirect_link,recommended_books,books_in_series,description
0,630104,Inner Circle,https://www.goodreads.com//book/show/630104.In...,(Private #5),https://i.gr-assets.com/images/S/compressed.ph...,"Kate Brian, Julian Peploe",https://www.goodreads.com/author/show/94091.Ka...,7597,196,4.03,3045,2323,1748,389,92,220.0,January 1st 2007,Simon Schuster Books for Young Readers,Inner Circle,"Young Adult 161, Mystery 45, Romance 32",1416950419,9781416950417,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"726458, 726458, 1537534, 3047848, 1651302, 304...","381489, 381501, 352428, 630103, 1783281, 17832...",Reed Brennan arrived at Easton Academy expecti...
1,9487,A Time to Embrace,https://www.goodreads.com//book/show/9487.A_Ti...,(Timeless Love #2),https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,https://www.goodreads.com/author/show/3159984....,4179,177,4.35,2255,1290,518,93,23,400.0,October 29th 2006,Thomas Nelson,NaN,"Christian Fiction 114, Christian 45, Fiction 3...",1595542329,9781595542328,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"127352, 127352, 40642197, 127353, 127354, 3891...",115036,"Ideje az Ã¶lelÃ©snek TÃ¶rtÃ©net a remÃ©nyrÅl,..."
2,6050894,Take Two,https://www.goodreads.com//book/show/6050894-t...,(Above the Line #2),https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,https://www.goodreads.com/author/show/3159984....,6288,218,4.23,3000,2020,1041,183,44,320.0,January 1st 2009,Zondervan,Take Two,"Christian Fiction 174, Christian 81, Fiction 58",NaN,NaN,NaN,"Bloomington, Indiana(United States)",NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"706250, 706250, 666481, 11942636, 706241, 1273...","4010795, 40792877, 7306261",Filmmakers Chase Ryan and Keith Ellison have c...
3,39030,Reliquary,https://www.goodreads.com//book/show/39030.Rel...,(Pendergast #2),https://i.gr-assets.com/images/S/compressed.ph...,"Douglas Preston, Lincoln Child",https://www.goodreads.com/author/show/12577.Do...,38382,1424,4.01,12711,15407,8511,1429,324,464.0,1997,Tor Books,Reliquary,"Thriller 626, Mystery 493, Horror 432, Fiction...",0765354950,9780765354952,NaN,"New York State(United States) New York City, N...","Aloysius X.L. Pendergast, Margo Green, William...",NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"39026, 39026, 11007668, 3986318, 51887330, 194...","67035, 39031, 39033, 136637, 136638, 30068, 39...",NaN
4,998,The Millionaire Next Door: The Surprising Secr...,https://www.goodreads.com//book/show/998.The_M...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,"Thomas J. Stanley, William D. Danko",https://www.goodreads.com/author/show/659.Thom...,72168,3217,4.04,27594,25219,14855,3414,1086,258.0,October 28th 1995,Gallery Books,The Millionaire Next Door: The Surprising Secr...,"Economics-Finance 1162, Nonfiction 910, Busine...",0671015206,9780671015206,NaN,NaN,NaN,Independent Publisher Book Award (IPPY) Nomine...,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"1052, 1052, 763362, 69571, 78427, 30186948, 10...",NaN,The incredible national bestseller that is cha...


In [3]:
def clean_data(df):
    df = df[df["rating_count"]>75000]
    df = df.drop(columns=['id', 'series', 'cover_link', 'author_link', 'rating_count', 'review_count', 'five_star_ratings',
       'four_star_ratings', 'three_star_ratings', 'two_star_ratings',
       'one_star_ratings', 'number_of_pages', 'date_published', 'publisher', 'original_title', 'isbn', 'isbn13', 'asin', 'settings', 'characters', 'awards', 'amazon_redirect_link',
       'worldcat_redirect_link', 'recommended_books', 'books_in_series'])
    df = df.dropna(subset=["description"])
    df = df.dropna(subset=["genre_and_votes"])
    df = df.drop_duplicates(subset=["title", "author"])
    df = df[~(df['description'].str.strip() == '')]
    
    def desc_length(row):
        return len(row.split())
    
    df["desc_length"] = df["description"].apply(desc_length)
    df = df[df["desc_length"] > 15]
    
    def is_english(text):
        return langid.classify(text)[0] == "en"

    df = df[df['description'].apply(is_english)]

    return df

In [4]:
df = clean_data(df)

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load pre-trained T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define the summarization function
def summarize_text(text):
    if isinstance(text, str) and len(text) > 0:  # Ensure text is not empty
        input_text = "summarize: " + text
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

        # Generate the summary
        summary_ids = model.generate(input_ids, max_length=100, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

        # Decode the summary
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary
    else:
        return ""


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
df['description'] = df.apply(
    lambda row: summarize_text(row['description']) if row['desc_length'] > 200 else row['description'],
    axis=1
)

In [9]:
df = df.drop(columns="desc_length")
df.head()

,title,link,author,average_rating,genre_and_votes,description
69,Catherine the Great: Portrait of a Woman,https://www.goodreads.com//book/show/10414941-...,Robert K. Massie,3.92,"History 1896, Biography 1631, Nonfiction 1325,...","born into minor German nobility, she transform..."
83,The Partner,https://www.goodreads.com//book/show/5350.The_...,John Grisham,3.92,"Fiction 1135, Thriller 473, Mystery 402, Thril...",they watched Danilo Silva for days before they...
135,The Interestings,https://www.goodreads.com//book/show/15815333-...,Meg Wolitzer,3.58,"Fiction 2009, Contemporary 278, Literary Ficti...",the interestings explores the meaning of talen...
154,Flight Behavior,https://www.goodreads.com//book/show/13438524-...,Barbara Kingsolver,3.79,"Fiction 1786, Literary Fiction 171, Contempora...",flight Behavior transfixes from its opening sc...
379,Don't Sweat the Small Stuff ... and It's All S...,https://www.goodreads.com//book/show/170548.Do...,Richard Carlson,3.98,"Self Help 830, Nonfiction 671, Psychology 178,...",Richard Carlson reveals ways to calm down in t...


In [1]:
#### EXTRACTING THEMES AND MOODS ####

In [ ]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-mpnet-base-v2")

kw_model = KeyBERT(embedding_model)

def extract_themes(text):
    if isinstance(text, str):  # Ensure text is valid
        themes = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', top_n=5)
        return [theme[0] for theme in themes]  # Extract only the keywords (ignore scores)
    return []

# Apply the function to the 'description' column
df["themes"] = df["description"].apply(extract_themes)

In [ ]:
from transformers import pipeline

# Load emotion analysis pipeline
emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

def extract_emotions(text):
    emo = []
    emotions = emotion_pipeline(text)
    emo.append(emotions[0][0]["label"])
    emo.append(emotions[0][1]["label"])
    return emo

df["emotions"] = df["description"].apply(extract_emotions)

Device set to use cpu


In [ ]:
#### EMBEDDING THEMES AND EMOTIONS ####

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Combine themes and emotions into a single string
df["features"] = df["themes"].apply(lambda x: " ".join(x)) + " " + df["emotions"].apply(lambda x: " ".join(x))

# Convert book features to embeddings
df["embedding"] = df["features"].apply(lambda x: embedding_model.encode(x, convert_to_tensor=True))

# Save embeddings for faster recommendation
df.to_pickle("books_with_embeddings.pkl")
